wiht conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)

這樣寫就不用close了，但sql3不支援 改成下面另一個寫法

In [ ]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql ='''
    CREATE TABLE record (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE (date,sitename)
    );
    '''
    with conn.cursor() as cursor:
        cursor.execute(sql)




以下寫法 也不用 寫commit 跟 close

In [5]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql ='''
    CREATE TABLE record (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE (date,sitename)
    );
    '''
    cursor =conn.cursor()
    cursor.execute(sql)

In [6]:
import requests

url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'

try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)

else:
    sitenames = set()

    for items in data['records']:
        sitenames.add(items['sitename'])

    sitenames = list(sitenames)


['馬祖', '竹東', '屏東（琉球）', '嘉義', '基隆', '彰化', '斗六', '潮州', '林口', '復興', '恆春', '臺東', '鳳山', '三重', '前金', '新竹', '陽明', '臺南', '崙背', '小港', '金門', '桃園', '高雄（湖內）', '左營', '前鎮', '竹山', '仁武', '萬華', '關山', '新北(樹林)', '安南', '新港', '大里', '永和', '汐止', '忠明', '屏東', '麥寮', '板橋', '花蓮', '臺南（麻豆）', '宜蘭（頭城）', '臺西', '林園', '新營', '新店', '馬公', '頭份', '大同', '三義', '豐原', '宜蘭', '埔里', '平鎮', '美濃', '屏東(枋山)', '冬山', '萬里', '士林', '中壢', '楠梓', '大城', '線西', '南投', '朴子', '沙鹿', '淡水', '大園', '苗栗', '彰化（員林）', '橋頭', '觀音', '富貴角', '湖口', '西屯', '菜寮', '松山', '大寮', '二林', '善化', '龍潭', '古亭', '土城', '新莊', '中山']


In [8]:
import datasouce
datasouce.get_selected_data(sitename='萬華')

[['2024-11-04 10:00', '臺北市', '37', '13', '良好', '25.046503', '121.507972'],
 ['2024-11-04 09:00', '臺北市', '33', '11', '良好', '25.046503', '121.507972'],
 ['2024-11-04 09:00', '臺北市', '33', '11', '良好', '25.046503', '121.507972'],
 ['2024-11-04 08:00', '臺北市', '34', '12', '良好', '25.046503', '121.507972'],
 ['2024-11-04 08:00', '臺北市', '34', '12', '良好', '25.046503', '121.507972'],
 ['2024-11-04 07:00', '臺北市', '32', '12', '良好', '25.046503', '121.507972'],
 ['2024-11-04 07:00', '臺北市', '32', '12', '良好', '25.046503', '121.507972'],
 ['2024-11-04 06:00', '臺北市', '31', '6', '良好', '25.046503', '121.507972'],
 ['2024-11-04 06:00', '臺北市', '31', '6', '良好', '25.046503', '121.507972'],
 ['2024-11-04 05:00', '臺北市', '34', '10', '良好', '25.046503', '121.507972'],
 ['2024-11-04 05:00', '臺北市', '34', '10', '良好', '25.046503', '121.507972']]

以下方法 insert or ignore 搭配 上面的UNIQUE()  
就可以避免重複輸入資料到資料庫

In [12]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql ='''
    INSERT OR IGNORE INTO record(sitename,county,aqi,status,pm25,date,lat,lon)
VALUES ('萬華', '臺北市', 37, '良好', 13, '2024-11-04 10:00',25.046503, 121.507972);
    '''
    cursor =conn.cursor()
    cursor.execute(sql)

In [14]:
import sqlite3
conn = sqlite3.connect("AQI.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'

try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
except Exception as e:
            print(e)

else:
    sitenames = set()
    with conn:
        cursor =conn.cursor() 
        for items in data['records']:
            sitenames = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] !='' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] !='' else 0
            date = items['datacreationdate']
            lat = float(items['latitude']) if items['latitude'] !='' else 0
            lon = float(items['longitude']) if items['longitude'] != '' else 0
            sql = '''
                INSERT OR IGNORE INTO record(sitename,county,aqi,status,pm25,date,lat,lon)
                VALUES (?, ?, ?, ?,?, ?,?,?);
                ''' 
            cursor.execute(sql,(sitenames,county,aqi,status,pm25,date,lat,lon))
